In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [2]:
from pyquil.quil import Program
import pyquil.api as api
from pyquil.gates import *
qvm = api.QVMConnection()

In [3]:
from grove.pyvqe.vqe import VQE
from scipy.optimize import minimize

In [4]:
from pyquil.paulis import sX, sY, sZ, ID

We are going to try to use VQE to calculate the minimum energy fo LiH. We will use VQE in Pyquil/Grove and use the Nelder-Mead Algorithm to start.

The first thing we are doing is representing the hamiltonian of this system in Pyquil.

In [5]:
hamiltonian = -0.096022*sZ(0) + -0.206128*sZ(0)*sZ(1) + 0.364746*sZ(1) + 0.096022*sZ(2) + -0.206128*sZ(2)*sZ(3) + -0.364746*sZ(3) + -0.145438*sZ(0)*sZ(2) + 0.056040*sZ(0)*sZ(2)*sZ(3) + 0.110811*sZ(0)*sZ(3) + -0.056040*sZ(0)*sZ(1)*sZ(2) + 0.080334*sZ(0)*sZ(1)*sZ(2)*sZ(3) + 0.063673*sZ(0)*sZ(1)*sZ(3) + 0.110811*sZ(1)*sZ(2) + -0.06373*sZ(1)*sZ(2)*sZ(3) + -0.095216*sZ(1)*sZ(3)
+ -0.012585*sX(0)*sZ(1) + 0.012585*sX(0) + 0.012585*sX(2)*sZ(3) + 0.012585*sX(2) + -0.002667*sX(0)*sZ(1)*sX(2)*sZ(3) + -0.002667*sX(0)*sZ(1)*sX(2) + 0.002667*sX(0)*sX(2)*sZ(3) + 0.002667*sX(0)*sX(2) + 0.007265*sX(0)*sZ(1)*sZ(3) + -0.007265*sX(0)*sZ(3) + 0.007265*sZ(1)*sX(2)*sZ(3) + 0.007265*sZ(1)*sX(2)
+ -0.029640*sX(0)*sX(1) + 0.002792*sX(1) + -0.029640*sX(2)*sX(3) + 0.002792*sX(3) + -0.008195*sX(0)*sX(2)*sX(3) + -0.001271*sX(0)*sX(3) + -0.008195*sX(0)*sX(1)*sX(2) + 0.028926*sX(0)*sX(1)*sX(2)*sX(3) + 0.007499*sX(0)*sX(1)*sX(3) + -0.001271*sX(1)*sX(2) + 0.007499*sX(1)*sX(2)*sX(3) + 0.009327*sX(1)*sX(3)
+ 0.029640*sY(0)*sY(1) + 0.029640*sY(2)*sY(3) + 0.028926*sY(0)*sY(1)*sY(2)*sY(3)
+ 0.002792*sZ(0)*sX(1) + -0.002792*sZ(2)*sX(3) + -0.016781*sZ(0)*sZ(2)*sX(3) + 0.016781*sZ(0)*sX(3) + -0.016781*sZ(0)*sX(1)*sZ(2) + -0.016781*sX(1)*sZ(2) + -0.009327*sZ(0)*sX(1)*sZ(2)*sX(3) + 0.009327*sZ(0)*sX(1)*sX(3) + -0.009327*sX(1)*sZ(2)*sX(3)
+ -0.011962*sZ(0)*sX(2)*sZ(3) + -0.011962*sZ(0)*sX(2) + 0.000247*sZ(0)*sZ(1)*sX(2)*sZ(3) + 0.000247*sZ(0)*sZ(1)*sX(2)
+ 0.039155*sZ(0)*sX(2)*sX(3) + -0.002895*sZ(0)*sZ(1)*sX(2)*sX(3) + -0.009769*sZ(0)*sZ(1)*sX(3) + -0.024280*sZ(1)*sX(2)*sX(3) + -0.008025*sZ(1)*sX(3)
+ -0.039155*sZ(0)*sY(2)*sY(3) + 0.002895*sZ(0)*sZ(1)*sY(2)*sY(3) + 0.024280*sZ(1)*sY(2)*sY(3)
+ -0.011962*sX(0)*sZ(1)*sZ(2) + 0.011962*sX(0)*sZ(2) + -0.000247*sX(0)*sZ(1)*sZ(2)*sZ(3) + 0.000247*sX(0)*sZ(2)*sZ(3)
+ 0.008195*sX(0)*sZ(1)*sX(2)*sX(3) + 0.0001271*sX(0)*sZ(1)*sX(3) + -0.007499*sX(0)*sX(1)*sZ(2)*sX(3)
+ -0.008195*sX(0)*sZ(1)*sY(2)*sY(3) + 0.008195*sX(0)*sY(2)*sY(3) + 0.007499*sY(0)*sY(1)*sZ(2)*sX(3)
+  -0.001271*sX(0)*sZ(1)*sZ(2)*sX(3) + 0.001271*sX(0)*sZ(2)*sX(3) + 0.008025*sZ(1)*sZ(2)*sX(3) + 0.009769*sZ(0)*sZ(1)*sZ(2)*sX(3)
+ -0.039155*sX(0)*sX(1)*sZ(2) + -0.002895*sX(0)*sX(1)*sZ(2)*sZ(3) + 0.024280*sX(0)*sX(1)*sZ(3) + -0.009769*sX(1)*sZ(2)*sZ(3) + 0.008025*sX(1)*sZ(3) + -0.001271*sZ(0)*sX(1)*sX(2)*sZ(3) + -0.001271*sZ(0)*sX(1)*sX(2) + 0.008025*sZ(0)*sX(1)*sZ(3)
+ 0.039155*sY(0)*sY(1)*sZ(2) + 0.002895*sY(0)*sY(1)*sZ(2)*sZ(3) + -0.024280*sY(0)*sY(1)*sZ(3) + 0.007499*sZ(0)*sX(1)*sX(2)*sX(3)
+ -0.008195*sX(0)*sX(1)*sX(2)*sZ(3) + -0.001271*sX(1)*sX(2)*sZ(3) + -0.007499*sZ(0)*sX(1)*sY(2)*sY(3)
+ 0.008195*sY(0)*sY(1)*sX(2)*sZ(3) + 0.008195*sY(0)*sY(1)*sX(2) + -0.009769*sZ(0)*sX(1)*sZ(2)*sZ(3)
+ -0.028926*sX(0)*sX(1)*sY(2)*sY(3) +  -0.007499*sX(1)*sY(2)*sY(3)  + -0.028926*sY(0)*sY(1)*sX(2)*sX(3) + -0.007499*sY(0)*sY(1)*sX(3)

We will now create a small ansatz function. This provides the gates that give us our states. We will then run our VQE using these gates to find the minimum energy.

In [6]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0), RY(params[1], 0), RZ(params[2],0),
                   RX(params[3], 1), RY(params[4], 1), RZ(params[5],1),
                   RX(params[6], 2), RY(params[7], 2), RZ(params[8],2),
                   RX(params[9], 3), RY(params[10], 3), RZ(params[11],3)
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14158201e+00,   7.80035253e-06,   4.49390667e+00,
        -8.79683447e-07,   3.14158407e+00,   3.14369079e+00,
        -3.76259573e-06,   2.66922549e-05,  -1.22785945e+00,
         8.96356157e-06,   3.93712138e-06,   1.96039179e+00]), 'fun': -0.83501499994339268}


Groundstate energy is approximately -0.835015, which agrees with what we saw using numpy.

What happens if we try different number of variational parameters? What happens if we only have X rotations (only RX gates in our small_ansatz function)? We will now test this out.

In [7]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1), 
                   RX(params[2], 2), 
                   RX(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14163563e+00,   3.14160580e+00,   5.72405212e-06,
        -3.42813130e-07]), 'fun': -0.83501499992146622}


A little bit farther from -0.835015. If we try this for only RZ, we get:

In [8]:
def smallish_ansatz_HXYZ(params):
    return Program(RZ(params[0], 0),
                   RZ(params[1], 1), 
                   RZ(params[2], 2), 
                   RZ(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([ 1.,  1.,  1.,  1.]), 'fun': -0.35101100000000007}


This minimum energy result is very far from what we expect. This indicates that the groundstate very closely represents some rotation about the x axis, involves very little rotation about the z axis. Let's try this for the T gate.

In [9]:
def smallish_ansatz_HXYZ(params):
    return Program(T(0),
                   T(1), 
                   T(2), 
                   T(3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([ 1.,  1.,  1.,  1.]), 'fun': -0.35101100000000007}


We see that we get the same (not great) result for T gates, since T gates are also just rotations about the Z axis, like RZ is.
Let's now try to see which qubit the X rotation is most important for by replacing the X rotation for one qubit with the Z rotation and repeating this four times (once for each qubit).

In [10]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1), 
                   RX(params[2], 2), 
                   RZ(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([ -3.14161865e+00,   3.14159715e+00,   4.01728510e-06,
         1.90948147e+00]), 'fun': -0.83501499998054673}


In [11]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1), 
                   RZ(params[2], 2), 
                   RX(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14161433e+00,   3.14158881e+00,   6.07231841e-01,
         2.31415454e-05]), 'fun': -0.83501499987013639}


In [12]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RZ(params[1], 1), 
                   RX(params[2], 2), 
                   RX(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  4.93867055e-06,   2.93047803e+00,  -1.70408453e-05,
        -1.44495610e-05]), 'fun': -0.35101099993530793}


In [13]:
def smallish_ansatz_HXYZ(params):
    return Program(RZ(params[0], 0),
                   RX(params[1], 1), 
                   RX(params[2], 2), 
                   RX(params[3], 3),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  9.85325676e-01,   3.14160517e+00,  -4.23067385e-05,
        -5.61947278e-06]), 'fun': -0.74791099973588415}


We see that the X rotation for the second and first qubit (mostly the second qubit) matter much much more than the X rotation about the 3rd and 4th qubit. In fact, the scenario with an X rotation for the first three qubits, and a Z rotation for the last qubit yields the best possible result.

In [14]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1), 
                   RX(params[2], 2),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14162753e+00,   3.14157891e+00,  -8.85630990e-06]), 'fun': -0.83501499993072792}


In [15]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([-3.14162319,  3.14157863]), 'fun': -0.83501499993695971}


We have confirmed that we only really need the x rotation about for the first two qubits. Also, sometimes when we give VQE too many parameters, it actually is less exact. We get closer to the true ground state energy if we just include two RX gates for the first two qubits than if we include RX gates for the first three qubits.